# Infobae

En esta notebook veremos qué se puede hacer con los artículos de infobae...

In [1]:
%load_ext autoreload
%autoreload 2
from mongoengine import connect
from tweepyrate import create_apps
from hatespeech_models import Tweet, Article

client = connect("hatespeech-news")
db = client["hatespeech-news"]

In [2]:
import json
with open("../../config/query_2.0.json") as f:
    query = json.load(f) 

query

{'users': ['LANACION', 'clarincom', 'cronica', 'infobae', 'perfilcom'],
 'min_date': '2020-02-10T00:00:00Z',
 'search_terms': ['coronavirus',
  'covid',
  'cuarentena',
  'normalidad',
  'aislamiento ',
  'padecimiento',
  'encierro',
  'fase',
  'infectados',
  'Wuhan',
  'distanciamiento',
  'fiebre',
  'síntomas',
  'dengue',
  'aedes',
  'mosquito',
  'cacharro']}

In [6]:
import datetime

min_date = datetime.datetime.strptime(query["min_date"], "%Y-%m-%dT%H:%M:%SZ")
text_query = " ".join(query["search_terms"])

articles = Article.objects(
    user="infobae",
    created_at__gte=min_date
).search_text(text_query).order_by('created_at')

articles.count()

8639

In [9]:
for art in articles.limit(30):
    print("="*80 +'\n\n')
    print(art.created_at, '\n\n')
    print(art.title, "\n\n")
    print(f"comentarios: {len(art['comments'])}")
    print(art.url, "\n\n")
    print(":"*30, '\n\n')
    print(art.body)



2020-02-10 00:03:59 


Daniel Osvaldo publicó un fuerte mensaje en defensa de Jimena Barón: “Te banco a muerte, no merecés esta m...” 


comentarios: 34
https://ibae.am/2SdMPhp 


:::::::::::::::::::::::::::::: 


"Si quieren ser ejemplo, prediquen amor", pidió Osvaldo a quienes criticaron a su ex mujer (Instagram)

La promoción del nuevo tema de Jimena Barón, “Puta”, que incluía la distribución de afiches en la vía pública que simulaban a los que se utilizan en la calle para ofrecer servicios sexuales, desató la polémica. La cantante recibió una fuerte respuesta negativa y el repudio del público y de la prensa, además de una denuncia de AVIVI (Asociación de Víctimas de Violación).

En medio del tenso momento, la cantante decidió suspender varios shows previstos, y su productora emitió un comunicado informando que Jimena había tenido que ser atendida por un psiquiatra que le recomendó reposo y alejamiento público.

Pero sorprendió la defensa que realizó Daniel Osvaldo de su ex mujer 

In [ ]:
from newspaper import Article as NSArticle
url = "https://www.infobae.com/teleshow/infoshow/2020/05/28/el-contundente-reclamo-de-dady-brieva-a-alberto-fernandez-tenes-que-hacerte-responsable/?utm_medium=Echobox&amp;utm_source=Twitter#Echobox=1590628593"
article = NSArticle(url, language="es")

article.download()
article.parse()

In [3]:
elem = article.doc.get_element_by_id("article-content")

elem.text_content??

Signature: elem.text_content()
Source:   
    def text_content(self):
        """
        Return the text content of the tag (and the text in any children).
        """
        return _collect_string_content(self)
File:      ~/.local/share/virtualenvs/hatespeech-collection-qaK7Gomm/lib/python3.6/site-packages/lxml/html/__init__.py
Type:      method


In [4]:
"""
Busco el elemento donde está todo el contenido
"""
elem = article.doc.get_element_by_id("article-content")

"""
Esto es código particular de infobae
Saco los links que están al pie de la nota
"""
children = list(elem.getchildren())

last = children[-1]

while len(list(children[-1].iter("a"))) > 0:
    children.pop(-1)
    
"""
Me quedo con el texto
"""
text_children = [t for t in children if t.tag not in ["meta", "script"]]
text = "\n\n".join(t.text_content().strip() for t in text_children)

print(text.strip())

El pedido de Dady Brieva a Alberto Fernández: "Tenés que hacerte responsable"



Rubén Enrique “Dady” Brieva le envió un mensaje enérgico al presidente Alberto Fernández desde su programa de radio, Volver Mejores, que se emite en El Destape. Fiel a su estilo, el actor analizó una tendencia que se instaló ayer en redes sociales (#EsAhoraAlberto) en donde se acumularon ideas y solicitudes al gobierno nacional de reformas radicales e intimó al jefe de Estado a avanzar con políticas públicas que -según consideró- son esperadas por los votantes del Frente de Todos. “Tenés que hacerte responsable”, le exigió.

El editorial incluyó un pedido para remover fiscales, encarcelar ex funcionarios, intervenir medios de comunicación y avanzar sobre la libertad de prensa. Y cerró con un mensaje sobre el futuro de la Argentina: “Sí o sí vamos a ser Venezuela”.

“Cuando volvimos, creí que no iban a estar más los que estaban. Creí que se iba a investigar. Me dio ilusión eso. Yo sé que no es fácil, pero n

## Noticias repetidas